In [1]:
import math
from sqlite3 import connect
import datetime as dt
from astropy.coordinates import SkyCoord, Angle
from astropy.time import Time
import astropy.units as u
import  astroplan as ap
import pandas as pd


In [2]:
def all_night_observing_session(observer: ap.Observer, utc_date: str) -> [Time, Time]:
    time = Time(utc_date, scale='tdb')
    return [observer.twilight_evening_civil(time, which='nearest'), observer.twilight_morning_civil(time, which='next')]

In [3]:
observer = ap.Observer.at_site("lbt")

observing_sessions = [
    # '2024-01-13',
    # '2024-01-14',
    # '2024-01-15',
    # '2024-01-16',
    # '2024-01-17',
    # '2024-01-18',
    # all_night_observing_session(observer, '2024-02-20'),
    all_night_observing_session(observer, '2024-02-21'),
]


In [82]:
conn = connect("db.sqlite3")

# read base list of targets
targets = pd.read_sql(
    """
        select *
        from tom_target t
        join tom_catalogassociation ca on ca.target_id = t.id and ca.catalog = 'TESS TICv8' and ca.association = 'Primary ID'
        join tom_tess_ticv8 tt on tt.Identifier = ca.catalog_id
        ;""",
    conn,
    index_col="target_id",
)

# get speckle and spectrum data and add it to main targets table
speckle = pd.read_sql("select * from tom_specklerawdata;", conn, index_col="target_id")
spectrum = pd.read_sql(
    "select * from tom_spectrumrawdata;", conn, index_col="target_id"
)

targets = targets.join(speckle.groupby("target_id").id.agg(num_speckle="count")).rename(
    columns={"num_speckle": "Num Speckle"}
)
targets = targets.join(
    spectrum.groupby("target_id").id.agg(num_spectra="count")
).rename(columns={"num_spectra": "Num Spectra"})
targets.fillna(0, inplace=True)
targets["Num Speckle"] = targets["Num Speckle"].astype(int)
targets["Num Spectra"] = targets["Num Spectra"].astype(int)

# add columns for membership in each target list
for (target_list,) in conn.execute("select name from tom_targetlist;").fetchall():
    list_members = [
        result[0]
        for result in conn.execute(
            """
            select t.local_id
            from tom_targetlist tl
            join tom_targetlist_targets tlt on tlt.targetlist_id = tl.id
            join tom_target t on t.id = tlt.target_id
            where tl.name = ?
            ;
            """,
            [target_list],
        ).fetchall()
    ]
    targets[target_list] = False
    targets.loc[targets.local_id.isin(list_members), target_list] = True

# add parameters for the binary systems
params = pd.read_sql(
    """
    select t.local_id, bp.*
    from tom_binaryparameters bp
    join tom_scienceresult sr on sr.id = bp.scienceresult_ptr_id
    join tom_target t on t.id = sr.target_id
    ;""",
    conn,
    index_col="local_id",
)

# TODO: following code only works for binary systems - generalize to n-member systems
joined_params = params.drop(columns=["scienceresult_ptr_id"])
joined_params = joined_params[joined_params["member"] == "A"].join(
    joined_params[joined_params["member"] == "B"],
    on="local_id",
    lsuffix="_A",
    rsuffix="_B",
)
targets = targets.join(joined_params, on="local_id", how="left")

# calculate eclipse phase timings
# TODO: following code only works for binary systems - generalize to n-member systems
#members = list(map(lambda name: name.replace("period_", "") if "period_" in name, targets.columns))
members = ["A", "B"]
components = ["a", "b"]
phase_milestones = [0.00, 0.25, 0.5, 0.75]
# some helper functions for consistent column naming
def phase_milestone_col(member, component, milestone):
    return f"{member}{component}_phase_{milestone:.2f}"
def eclipse_timing_col(member, component, event):
    return f"{member}{component}_{event}"
def entire_eclipse_col(member, component=""):
    return f"{member}{component}_entire_eclipse"
for member in members:
    targets[entire_eclipse_col(member)] = False
    for milestone in phase_milestones:
        targets[phase_milestone_col(member, "", milestone)] = False
    for component in components:
        targets[entire_eclipse_col(member, component)] = False
        targets[eclipse_timing_col(member, component, "ingress")] = False
        targets[eclipse_timing_col(member, component, "egress")] = False
        for milestone in phase_milestones:
            targets[phase_milestone_col(member, component, milestone)] = False

# TODO: following code only handles the first observing session - generalize to all sessions
t_beg = observing_sessions[0][0].jd
t_end = observing_sessions[0][1].jd
for (index, row) in targets.iterrows():
    for member in members:
        period_col = f"period_{member}"
        if not period_col in row or math.isnan(period := row[period_col]):
            continue
        num_periods = (t_end - t_beg) / period
        targets.loc[index, f"num_periods_{member}"] = num_periods
        member_components = []
        for component in components:
            component_name = f"t0_{component}_{member}" #TODO: fix inconsistent column name
            if not component_name in row or math.isnan(t0 := row[component_name]):
                continue
            member_components.append(component)
            t_pre = math.floor((t_beg - t0) / period) * period + t0
            t_post = math.ceil((t_end - t0) / period) * period + t0
            beg_phase = (t_beg - t_pre) / period
            end_phase = 1 - (t_post - t_end) / period
            targets.loc[index, f"beg_phase_{member}{component}"] = beg_phase
            targets.loc[index, f"end_phase_{member}{component}"] = end_phase
            duration_jd = row[f"duration_{component}_{member}"] / 24 #TODO: fix inconsistent column name
            ingress = t_beg < t_pre + period - duration_jd < t_end
            targets.loc[index, eclipse_timing_col(member, component, "ingress")] = ingress
            egress = t_beg < t_pre + period + duration_jd < t_end
            targets.loc[index, eclipse_timing_col(member, component, "egress")] = egress
            targets.loc[index, entire_eclipse_col(member, component)] = ingress & egress
            # mark any phase milestones
            for milestone in phase_milestones:
                targets.loc[index, phase_milestone_col(member, component, milestone)] = beg_phase < milestone < end_phase
            # phase = 0 needs to be handled differently because of the wraparound issue
            targets.loc[index, phase_milestone_col(member, component, 0)] = (num_periods > 1) | (end_phase < beg_phase)
        targets.loc[index, entire_eclipse_col(member)] = targets.loc[index, [entire_eclipse_col(member, component) for component in member_components]].any()
        for milestone in phase_milestones:
            targets.loc[index, phase_milestone_col(member, "", milestone)] = targets.loc[index, [phase_milestone_col(member, component, milestone) for component in member_components]].any()
targets["any_entire_eclipse"] = targets[[entire_eclipse_col(member) for member in members]].any(axis=1)
targets["any_ingress"] = targets[[eclipse_timing_col(member, component, "ingress") for member in members for component in components]].any(axis=1)
targets["any_egress"] = targets[[eclipse_timing_col(member, component, "egress") for member in members for component in components]].any(axis=1)
for milestone in phase_milestones:
    targets[phase_milestone_col("any", "", milestone)] = targets[[phase_milestone_col(member, component, milestone) for member in members for component in components]].any(axis=1)

targets

,id,local_id,source,target_type,id,catalog,catalog_id,association,id,version,...,end_phase_Ba,beg_phase_Bb,end_phase_Bb,any_entire_eclipse,any_ingress,any_egress,any_phase_0.00,any_phase_0.25,any_phase_0.50,any_phase_0.75
target_id,,,,,,,,,,,,,,,,,,,,,
3366,3366,TIC 270360534,Kostov 2023 arXiv:2309.14200,QuadEB,1024,TESS TICv8,270360534,Primary ID,236,20190415,...,NaN,NaN,NaN,False,False,False,False,False,False,False
3367,3367,TIC 219469945,Kostov 2022 arXiv:2202.05790,QuadEB,1025,TESS TICv8,219469945,Primary ID,323,20190415,...,0.839236,0.323834,0.357136,False,True,False,True,False,True,False
3368,3368,TIC 20212631,Kostov 2023 arXiv:2309.14200,QuadEB,1026,TESS TICv8,20212631,Primary ID,310,20190415,...,0.751309,NaN,NaN,True,True,True,True,False,False,True
3369,3369,TIC 150055835,Kostov 2023 arXiv:2309.14200,QuadEB,1027,TESS TICv8,150055835,Primary ID,389,20190415,...,0.166617,0.569050,0.605206,False,True,False,True,False,True,False
3370,3370,TIC 161043618,Kostov 2022 arXiv:2202.05790,QuadEB,1028,TESS TICv8,161043618,Primary ID,215,20190415,...,0.297335,0.459614,0.794435,False,False,True,True,False,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3559,3559,TIC 414969157,Kostov 2022 arXiv:2202.05790,QuadEB,1217,TESS TICv8,414969157,Primary ID,298,20190415,...,0.884037,0.340310,0.412237,False,False,False,False,False,False,False
3560,3560,TIC 27543409,Kostov 2022 arXiv:2202.05790,QuadEB,1218,TESS TICv8,27543409,Primary ID,387,20190415,...,0.333928,NaN,NaN,False,False,False,False,True,False,True
3561,3561,TIC 139914081,Kostov 2023 arXiv:2309.14200,QuadEB,1219,TESS TICv8,139914081,Primary ID,235,20190415,...,0.409466,0.713746,0.745296,True,True,True,True,False,False,False


In [83]:
#make astroplan objects for each target
fixed_targets = [
    ap.FixedTarget(
        coord=SkyCoord(
            frame="icrs",
            obstime=Time("2000.0", format="jyear", scale="tdb"),
            ra=target["ra"] * u.deg,
            dec=target["dec"] * u.deg,
            pm_ra_cosdec=target["pmRA"] * u.mas / u.yr,
            pm_dec=target["pmDEC"] * u.mas / u.yr,
            # distance=target["plx"] * u.pc,
        ),
        name=target["local_id"],
    )
    for (_, target) in targets.iterrows()
]

constraints = [
    ap.AltitudeConstraint(30*u.deg, 80*u.deg),
    ap.AirmassConstraint(2),
    # ap.AtNightConstraint.twilight_civil(),
    ]

#add a column to table for observability of each target
targets["Observable Nights"] = 0
for observing_session in observing_sessions:
    observable_tonight = ap.is_observable(constraints, observer, fixed_targets, observing_session)
    targets.loc[observable_tonight, "Observable Nights"] += 1

#TODO: add columns for beginning, ending, and max airmass

In [84]:
cols_basic = ["local_id", "ra", "GAIAmag"]
cols_rawdata = ["Num Speckle", "Num Spectra"]
cols_ephem = ["period_A", "period_B"]
cols_eclipse = [f"{member}_entire_eclipse" for member in members]
cols_rv = [f"{member}{component}_phase_{milestone}" for member in members for component in ["a", "b"] for milestone in [0.25, 0.75]]
cols_common = cols_basic + cols_rawdata + cols_ephem


In [85]:
# criteria={
#     # "test": ((targets["HQND"]), ""),
#     "Other Bright with spectra and speckle": ((targets["magnitude"] < 11) & (targets["Num Speckle"] > 0) & (targets["Num Spectra"] > 0) & (targets["HQND"] == False), "Low"),
#     "HQND, Dim": ((targets["HQND"]) & (targets["magnitude"]>13), "Low"),
#     "HQND Bright no spectra": ((targets["HQND"]) & (targets["Num Spectra"] == 0) & (targets["magnitude"] < 13), "Medium"),
#     "HQND Bright with spectra": ((targets["HQND"]) & (targets["Num Spectra"] > 0) & (targets["magnitude"] < 13), "High"),
#     "Featured": ((targets["Featured targets"]), "Highest"),
# }

min_per, max_per = 1.5, 5

criteria = {
    "Bright": (targets["GAIAmag"] < 19, "Low"),
    "Light curve": (targets["any_entire_eclipse"], "Medium"),
    "Best RV": (targets["any_phase_0.25"] | targets["any_phase_0.75"], "Highest"),
    # "Between eclipses": ((targets["phases_A"] > 1) | (targets["phases_B"] > 1), "High"),
    # "VATT Test": ((targets["period_A"] > max_per) & (targets["period_B"] < min_per) & (targets["period_B"] > 0)
    #     | (targets["period_A"] < min_per) & (targets["period_B"] > max_per) & (targets["period_A"] > 0), "Highest"),
}

matching_ids = {}
combined_ids = set()
targets["Priority"] = ""
targets["Criterion"] = ""
for name, (criterion, priority) in criteria.items():
    filtered_ids = set(targets[criterion  & (targets["Observable Nights"] > 0)]["local_id"])
    targets.loc[targets.local_id.isin(filtered_ids), "Priority"] = priority
    targets.loc[targets.local_id.isin(filtered_ids), "Criterion"] = name
    print(f"{len(filtered_ids):3d} targets from criterion: {name}")
    combined_ids=combined_ids | filtered_ids
    matching_ids[name] = filtered_ids
print(f"Total of {len(combined_ids)} targets")

observing_list = targets[targets["local_id"].isin(combined_ids)]
observing_list[cols_common + cols_rv + ["Criterion"]].sort_values("GAIAmag")

101 targets from criterion: Bright
 15 targets from criterion: Light curve
 58 targets from criterion: Best RV
Total of 101 targets


,local_id,ra,GAIAmag,Num Speckle,Num Spectra,period_A,period_B,Aa_phase_0.25,Aa_phase_0.75,Ab_phase_0.25,Ab_phase_0.75,Ba_phase_0.25,Ba_phase_0.75,Bb_phase_0.25,Bb_phase_0.75,Criterion
target_id,,,,,,,,,,,,,,,,
3398,TIC 344541836,317.850729,7.86518,7,0,2.409932,2.755276,False,True,True,False,True,False,False,True,Best RV
3375,TIC 367448265,78.382438,7.87389,3,2,0.418238,1.865520,False,True,False,False,True,False,False,True,Best RV
3439,TIC 153406662,93.228955,7.92700,0,0,2.521312,8.863059,False,False,False,False,False,False,False,False,Bright
3406,TIC 266395331,60.882924,8.75267,0,0,3.048970,5.239560,False,True,True,False,True,False,False,False,Best RV
3449,TIC 238558210,91.091266,9.25867,0,0,0.972387,5.337670,False,False,False,False,False,False,False,False,Bright
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3416,TIC 13021681,75.534801,14.77170,0,0,0.540612,11.065090,False,False,False,False,False,False,False,False,Light curve
3462,TIC 354314226,27.619654,15.01770,0,0,1.388452,1.842284,False,False,False,False,False,False,False,False,Light curve
3394,TIC 358422952,317.232893,15.08380,0,0,3.053610,3.075650,False,False,False,False,False,False,True,False,Best RV


In [ ]:
#exposure data from Robbie's LBT observing readme, with dupes removed manually
import numpy as np
mag=np.array([7.9,9.4,9.5,10,10.2,10.3,10.6,10.7,10.8,10.9,11,11.1,11.2,11.3,11.4,11.5,11.7,11.8,11.9,12,12.1,12.3,12.4,12.5,12.6,12.7,12.8,13.2,13.3,13.4,13.5,13.6,13.7,13.8,13.9,])
sec=np.array([60,60,60,60,60,60,60,60,60,72,72,84,78,84,102,102,132,138,150,192,180,240,270,258,270,318,360,510,594,630,666,726,768,858,918,])

from scipy.interpolate import CubicSpline
cs = CubicSpline(mag, sec)

#print out results in LBT readme format
#NOTE: targets are printed in ascending RA.  Rotate the list so first observable target is first.
print("TargetName       RA (J2000)     DEC (J2000)       Gmag     pmRA     pmDEC     Mode               ExposureTime   ExecutionTime        Priority  Notes")
print("-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------")
for _,target in targets[targets["local_id"].isin(combined_ids)].sort_values("ra").iterrows():
    exposure = cs(target["GAIAmag"])
    readout_sec = exposure + 140
    readout_min = readout_sec / 60
    print(
        f'{target["local_id"]:<15s} '
        f'{Angle(target["ra"]*u.deg).to_string(unit=u.hour,decimal=False,precision=2,sep=":"):>11s}    '
        f'{Angle(target["dec"]*u.deg).to_string(unit=u.deg,decimal=False,precision=2,sep=":",alwayssign=True):>12s}      '
        f'{target["GAIAmag"]:5.2f}    '
        f'{target["pmRA"]:6.2f}   '
        f'{target["pmDEC"]:6.2f}     '
        f'F300/CDII/CDVI     '
        f'1x{exposure:4.0f}sec     '
        f'{readout_sec:5.0f} sec = {readout_min:4.1f}min   '
        f'{target["Priority"]:<7s}   '
        "--"
    )

TargetName       RA (J2000)     DEC (J2000)       Gmag     pmRA     pmDEC     Mode               ExposureTime   ExecutionTime        Priority  Notes
-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
TIC 283940788    0:35:24.34    +62:54:05.74      11.73     -2.73    -0.15     F300/CDII/CDVI     1x 135sec       275 sec =  4.6min   Highest   --
TIC 266395331    4:03:31.90    +48:33:51.24       8.75     21.04   -28.05     F300/CDII/CDVI     1x  60sec       200 sec =  3.3min   Lowest    --
TIC 459959916    4:45:57.52     +4:49:46.63      13.23      2.92     5.88     F300/CDII/CDVI     1x 536sec       676 sec = 11.3min   Highest   --
TIC 13021681     5:02:08.35    -24:03:41.91      14.77     -2.86     4.03     F300/CDII/CDVI     1x-18787sec     -18647 sec = -310.8min   Highest   --
TIC 367448265    5:13:31.79    +35:39:10.99       7.87     -5.86    -3.43   

In [ ]:
#output targets in LBT obs.txt format
#NOTE: targets are printed in ascending RA.  Rotate the list so first observable target is first.
print("#TargetName      RA(J2000)      DEC(J2000)        VMag      MODE               t_exp")
for _,target in targets[targets["local_id"].isin(combined_ids)].sort_values("ra").iterrows():
    exposure = cs(target["GAIAmag"])
    readout_sec = exposure + 140
    readout_min = readout_sec / 60
    print(
        f'{target["local_id"]:<15s}  '
        f'{Angle(target["ra"]*u.deg).to_string(unit=u.hour,decimal=False,precision=2,sep=":"):>11s}    '
        f'{Angle(target["dec"]*u.deg).to_string(unit=u.deg,decimal=False,precision=2,sep=":",alwayssign=True):>12s}      '
        f'{target["GAIAmag"]:5.2f}     '
        f'F300/CDII/CDVI     '
        f'1x{exposure:4.0f}sec     '
    )

#TargetName      RA(J2000)      DEC(J2000)        VMag      MODE               t_exp
TIC 283940788     0:35:24.34    +62:54:05.74      11.73     F300/CDII/CDVI     1x 135sec     
TIC 266395331     4:03:31.90    +48:33:51.24       8.75     F300/CDII/CDVI     1x  60sec     
TIC 459959916     4:45:57.52     +4:49:46.63      13.23     F300/CDII/CDVI     1x 536sec     
TIC 13021681      5:02:08.35    -24:03:41.91      14.77     F300/CDII/CDVI     1x-18787sec     
TIC 367448265     5:13:31.79    +35:39:10.99       7.87     F300/CDII/CDVI     1x  60sec     
TIC 24700485      5:24:49.11     -7:35:32.49      12.53     F300/CDII/CDVI     1x 257sec     
TIC 238558210     6:04:21.90    +20:32:03.26       9.26     F300/CDII/CDVI     1x  60sec     
TIC 153406662     6:12:54.95     +9:02:08.86       7.93     F300/CDII/CDVI     1x  60sec     
TIC 36439758      6:48:56.43     -0:27:52.97      13.03     F300/CDII/CDVI     1x 418sec     
TIC 20212631     15:08:09.13    +39:58:12.86      10.52     F300/CD